# 0. Data

In [15]:
import random
import pandas as pd
import numpy as np
from datetime import datetime
import gc
import torch
import torch.nn as nn
random_seed=2022
torch.manual_seed(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)
torch.cuda.manual_seed(random_seed)

device = torch.device("cpu")
# device = torch.device("cuda")
from sklearn.model_selection import train_test_split

# CV
from sklearn.model_selection import ParameterGrid # creates a parameter grid to search for
from sklearn.model_selection import KFold # splits data into k-fold train and valid set

# done

## split data
- 'first 21 columns are input variables and last 7 columns are output variables'

In [2]:
# train_data = pd.read_csv('./data/SARCOSTst.csv', header=None)
# test_data = pd.read_csv('./data/SARCOSTrn.csv', header=None)

# X_train, X_valid, Y_train, Y_valid = train_test_split(train_data.iloc[:,:21], train_data.iloc[:,-7:], test_size=0.2)
# X_test, Y_test = test_data.iloc[:, :21], test_data.iloc[:, -7:]

# X_train = torch.tensor(X_train.values).to(device)
# Y_train = torch.tensor(Y_train.values).to(device)
# X_valid = torch.tensor(X_valid.values).to(device)
# Y_valid = torch.tensor(Y_valid.values).to(device)
# X_test = torch.tensor(X_test.values).to(device)
# Y_test = torch.tensor(Y_test.values).to(device)

# print('train set: ', X_train.shape, Y_train.shape)
# print('valid set: ', X_valid.shape, Y_valid.shape)
# print('test set: ', X_test.shape, Y_test.shape)


train set:  torch.Size([31587, 21]) torch.Size([31587, 7])
valid set:  torch.Size([7897, 21]) torch.Size([7897, 7])
test set:  torch.Size([5000, 21]) torch.Size([5000, 7])


In [16]:
# in case of using cross-validation

train_data = pd.read_csv('./data/SARCOSTst.csv', header=None)
test_data = pd.read_csv('./data/SARCOSTrn.csv', header=None)

X_train, Y_train = train_data.iloc[:,:21], train_data.iloc[:,-7:]
X_test, Y_test = test_data.iloc[:,:21], test_data.iloc[:,-7:]

X_train = torch.tensor(X_train.values).to(device)
Y_train = torch.tensor(Y_train.values).to(device)
X_test = torch.tensor(X_test.values).to(device)
Y_test = torch.tensor(Y_test.values).to(device)

print('train set: ', X_train.shape, Y_train.shape)
print('test set: ', X_test.shape, Y_test.shape)

train set:  torch.Size([39484, 21]) torch.Size([39484, 7])
test set:  torch.Size([5000, 21]) torch.Size([5000, 7])


# 1. Kernel

In [17]:
class LinearKernel:
    """
    standard dot product kernel k(a,b) = a^\top b
    :input: X1 (N*D), X2 (M*D)
    :output: covariance matrix (N*M)
    """
    def __init__(self):
        pass

    def __call__(self, X1, X2):
        # return torch.tensor([[torch.dot(x1, x2) for x1 in X1] for x2 in X2]) 
        return X1 @ X2.T


In [18]:
class GaussianKernel:
    """
    isotropic Gaussian kernel
    :input: X1 (N*D), X2 (M*D)
    :output: covariance matrix (N*M)
    """
    def __init__(self, sigma_k):
        self.sigma_k = sigma_k # isotropic Gaussian kernel variance 

    def __call__(self, X1, X2):
        # return np.exp(-(np.sum(X1**2, axis=1).values.reshape(-1, 1) +
        #                 np.sum(X2**2, axis=1).values.reshape(1, -1) - 2*X1@X2.T) / pow(self.sigma_k, 2))
        return torch.exp(-(torch.sum(X1**2, axis=1).reshape(-1, 1) +
                           torch.sum(X2**2, axis=1).reshape(1, -1) - 2*X1@X2.T) / pow(self.sigma_k, 2))


In [19]:
class SigmoidKernel:
    """
    hyperbolic tangent kernel
    :input: X1 (N*D), X2 (M*D)
    :output: covariance matrix (N*M)
    """
    def __init__(self, alpha):
        self.alpha = alpha 
    
    def __call__(self, X1, X2):
        return torch.tanh(self.alpha * X1 @ X2.T)

# 2. GP regression

## full model

In [92]:
class GP_Regression(nn.Module):
    def __init__(self, K, sigma_n, device):
        super().__init__()
        self.K = K
        self.sigma_n = nn.Parameter(torch.tensor(sigma_n), requires_grad=True)  # noise variance (Hyperparameter)
        self.device = device
        self.Sigma = torch.diag(torch.ones(7)) * self.sigma_n # output dim (=7)
    
    def fit(self, X_train, Y_train, X_test):
        self.X_train = X_train
        self.Y_train = Y_train
        self.X_test = X_test
        self.N = X_train.shape[0]
        self.M = X_test.shape[0]
        self.D = Y_train.shape[1] # output dim (=7)
        self.I = torch.eye(self.N)
        # sufficient statistics
        self.K_X_X = self.K(self.X_train, self.X_train)
        self.K_X_X = torch.block_diag(*[self.K_X_X]*self.D) # coregionalisation matrix (C) is an identity matrix with DxD
        self.K_Xt_X = self.K(self.X_test, self.X_train)
        self.K_Xt_X = torch.block_diag(*[self.K_Xt_X]*self.D)
        self.K_X_Xt = self.K_Xt_X.T
        self.K_Xt_Xt = self.K(self.X_test, self.X_test)
        self.K_Xt_Xt = torch.block_diag(*[self.K_Xt_Xt]*self.D)
        self.vec_Y = self.Y_train.T.reshape(self.D*self.N) # Y concat

    def predict(self):
        # calculate predictive mean
        mean = self.K_Xt_X @ torch.linalg.inv( self.K_X_X + torch.kron(self.Sigma, self.I).to(self.device) ) @ self.vec_Y
        return mean.reshape(self.D, -1).T # to compare with Y_test

    def __NLL_term_1__(self):
        return -0.5*(self.M*self.D) * torch.log(torch.tensor([2*torch.pi]))

    def __NLL_term_2__(self, CR): # Omega <- I
        Sigma = torch.diag(torch.ones(self.D)) * self.sigma_n
        SI = torch.kron(Sigma, self.I).to(self.device) 
        K = CR + SI
        return -0.5 * torch.log(torch.det(K))

    def __NLL_term_3__(self, CR): # Omega <- I
        Sigma = torch.diag(torch.ones(self.D)) * self.sigma_n
        SI = torch.kron(Sigma, self.I).to(self.device)
        K = CR + SI
        vec_Y = self.Y_train.T.reshape(self.D*self.N)  # Y concat
        return -0.5 * vec_Y.T @ torch.linalg.inv(K) @ vec_Y
    
    def calculate_NLL(self):
        K_X_X = self.K(self.X_train, self.X_train)
        CR = torch.block_diag(*[K_X_X]*self.D) 
        return self.__NLL_term_1__().to(self.device) + self.__NLL_term_2__(CR).to(self.device) + self.__NLL_term_3__(CR).to(self.device)


In [93]:
model = GP_Regression(K=GaussianKernel(sigma_k=1), sigma_n=0.05, device=device)
model.fit(X_train, Y_train, X_test)
pred = model.predict()
mse = nn.MSELoss()(pred, Y_test).item()
print(mse)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 611122612352 bytes.

## SOR approximation

In [20]:
class GP_Regression_SOR(nn.Module):
    def __init__(self, K, sigma_n, device, B):
        super().__init__()
        self.K = K
        self.sigma_n = sigma_n  # noise variance (Hyperparameter)
        self.device = device
        self.Sigma = torch.diag(torch.ones(7)) * self.sigma_n # output dim (=7)
        self.B = B # random sample portion (e.g., B=0.1 means 10% of data)
    
    def fit(self, X_train, Y_train, X_test):
        self.X_train = X_train
        self.Y_train = Y_train
        self.X_test = X_test
        self.N = X_train.shape[0]
        self.M = X_test.shape[0]
        self.D = Y_train.shape[1] # output dim (=7)
        self.I = torch.eye(self.N)
        self.B_train = X_train[torch.randperm(self.N)[:int(self.N*self.B)]]#.to(self.device)
        # sufficient statistics
        self.K_Xt_B = self.K(self.X_test, self.B_train)
        self.K_B_X = self.K(self.B_train, self.X_train)
        self.K_X_B = self.K_B_X.T
        self.K_B_B = self.K(self.B_train, self.B_train)
        self.vec_Y = self.Y_train.T.reshape(self.D*self.N) # Y concat

    def predict(self):
        # calculate predictive mean
        term_1 = torch.block_diag(*[self.K_Xt_B]*self.D)
        # term_2 = torch.linalg.inv(
        #     torch.block_diag(*[self.K_B_X @ self.K_X_B]*self.D) + torch.kron(self.Sigma, self.K_B_B).to(self.device)
        # )
        term_2 = torch.linalg.inv(
            torch.kron(torch.diag(torch.ones(self.D)), self.K_B_X@self.K_X_B) + torch.kron(self.Sigma, self.K_B_B)
        )
        term_3 = torch.block_diag(*[self.K_B_X]*self.D) @ self.vec_Y
        mean = term_1 @ term_2 @ term_3 
        return mean.reshape(self.D, -1).T # to compare with Y_test


In [7]:
model = GP_Regression_SOR(K=GaussianKernel(sigma_k=1), sigma_n=0.05, device=device, B=0.01) # 10%에 10분
model.fit(X_train, Y_train, X_test)
pred = model.predict()
mse = nn.MSELoss()(pred, Y_test).item()
print(mse)

360.07234805651103


# HPO

## Linear kernel

In [21]:
grid = {'sigma_n': [0.05, 0.1, 1, 5, 10]}
grid_list = list(ParameterGrid(grid))
grid_list

[{'sigma_n': 0.05},
 {'sigma_n': 0.1},
 {'sigma_n': 1},
 {'sigma_n': 5},
 {'sigma_n': 10}]

In [10]:
# for case in grid_list:

#     model = GP_Regression_SOR(K=LinearKernel(), sigma_n=case['sigma_n'], device=device, B=0.1)
#     start_time = datetime.now()
#     model.fit(X_train, Y_train, X_valid)
#     pred = model.predict()
#     runtime = datetime.now() - start_time
#     mse = nn.MSELoss()(pred, Y_valid).item()
    
#     print(case)
#     print('Runtime:: ', runtime.microseconds)
#     print('MSE loss: ', mse)
#     print('')


{'sigma_n': 0.05}
Runtime::  373195
MSE loss:  13070863.921733147

{'sigma_n': 0.1}
Runtime::  673963
MSE loss:  11031468.943707984

{'sigma_n': 1}
Runtime::  310300
MSE loss:  60396.19706776355

{'sigma_n': 5}
Runtime::  553296
MSE loss:  288277964.1572634

{'sigma_n': 10}
Runtime::  895313
MSE loss:  186614.25133434022



In [9]:
# cross-validation

for case in grid_list:

    X, Y = X_train, Y_train
    KF = KFold(n_splits=5, shuffle=True, random_state=2022)
    mse_list = []
    start_time = datetime.now()
    for train_idx, valid_idx in KF.split(X):
        X_train, Y_train = X[train_idx], Y[train_idx]
        X_valid, Y_valid = X[valid_idx], Y[valid_idx]
        # model
        model = GP_Regression_SOR(K=LinearKernel(), sigma_n=case['sigma_n'], device=device, B=0.1)
        model.fit(X_train, Y_train, X_valid)
        pred = model.predict()
        mse = nn.MSELoss()(pred, Y_valid).item()
        gc.collect(); torch.cuda.empty_cache() # clear memory
        mse_list.append(mse)
    final_loss = np.mean(mse_list)
    runtime = datetime.now() - start_time

    print(case)
    print('Runtime:: ', runtime.microseconds)
    print(final_loss)
    print('')


{'sigma_n': 0.05}
500430.33435679396

{'sigma_n': 0.1}
3946152.080139412

{'sigma_n': 1}
150596273.3726202

{'sigma_n': 5}
3000047.9525331864

{'sigma_n': 10}
1882629.2845048755



## Gaussian kernel

In [22]:
grid = {'sigma_n': [0.05, 0.1, 1, 5, 10],
        'sigma_k': [0.05, 1, 5]}
grid_list = list(ParameterGrid(grid))
grid_list

[{'sigma_k': 0.05, 'sigma_n': 0.05},
 {'sigma_k': 0.05, 'sigma_n': 0.1},
 {'sigma_k': 0.05, 'sigma_n': 1},
 {'sigma_k': 0.05, 'sigma_n': 5},
 {'sigma_k': 0.05, 'sigma_n': 10},
 {'sigma_k': 1, 'sigma_n': 0.05},
 {'sigma_k': 1, 'sigma_n': 0.1},
 {'sigma_k': 1, 'sigma_n': 1},
 {'sigma_k': 1, 'sigma_n': 5},
 {'sigma_k': 1, 'sigma_n': 10},
 {'sigma_k': 5, 'sigma_n': 0.05},
 {'sigma_k': 5, 'sigma_n': 0.1},
 {'sigma_k': 5, 'sigma_n': 1},
 {'sigma_k': 5, 'sigma_n': 5},
 {'sigma_k': 5, 'sigma_n': 10}]

In [23]:
for case in grid_list:

    model = GP_Regression_SOR(K=GaussianKernel(sigma_k=case['sigma_k']), sigma_n=case['sigma_n'], device=device, B=0.1)
    start_time = datetime.now()
    model.fit(X_train, Y_train, X_valid)
    pred = model.predict()
    runtime = datetime.now() - start_time
    mse = nn.MSELoss()(pred, Y_valid).item()
    
    print(case)
    print('Runtime:: ', runtime.microseconds)
    print('MSE loss: ', mse)
    print('')

KeyboardInterrupt: 

In [24]:
# cross-validation

for case in grid_list:

    X, Y = X_train, Y_train
    KF = KFold(n_splits=5, shuffle=True, random_state=2022)
    mse_list = []
    start_time = datetime.now()
    for train_idx, valid_idx in KF.split(X):
        X_train, Y_train = X[train_idx], Y[train_idx]
        X_valid, Y_valid = X[valid_idx], Y[valid_idx]
        # model
        model = GP_Regression_SOR(K=GaussianKernel(sigma_k=case['sigma_k']), sigma_n=case['sigma_n'], device=device, B=0.1)
        model.fit(X_train, Y_train, X_valid)
        pred = model.predict()
        mse = nn.MSELoss()(pred, Y_valid).item()
        gc.collect(); torch.cuda.empty_cache() # clear memory
        mse_list.append(mse)
    final_loss = np.mean(mse_list)
    runtime = datetime.now() - start_time

    print(case)
    print('Runtime:: ', runtime.microseconds)
    print(final_loss)
    print('')


{'sigma_k': 0.05, 'sigma_n': 0.05}
Runtime::  337810
373.6529561869671

{'sigma_k': 0.05, 'sigma_n': 0.1}
Runtime::  663101
372.86271657262324

{'sigma_k': 0.05, 'sigma_n': 1}
Runtime::  445649
372.2922511038711

{'sigma_k': 0.05, 'sigma_n': 5}
Runtime::  145060
374.9640591895366

{'sigma_k': 0.05, 'sigma_n': 10}
Runtime::  566051
377.20190016192936

{'sigma_k': 1, 'sigma_n': 0.05}
Runtime::  292263
353.11032374130866

{'sigma_k': 1, 'sigma_n': 0.1}
Runtime::  974394
355.90488723847614

{'sigma_k': 1, 'sigma_n': 1}
Runtime::  998744
357.3220117054649

{'sigma_k': 1, 'sigma_n': 5}
Runtime::  294131
363.9838755181798

{'sigma_k': 1, 'sigma_n': 10}
Runtime::  585363
369.5308261040593

{'sigma_k': 5, 'sigma_n': 0.05}
Runtime::  766372
157.61386283562814

{'sigma_k': 5, 'sigma_n': 0.1}
Runtime::  311069
168.8990889584535

{'sigma_k': 5, 'sigma_n': 1}
Runtime::  520360
199.70875205430255

{'sigma_k': 5, 'sigma_n': 5}
Runtime::  965865
248.90015127363637

{'sigma_k': 5, 'sigma_n': 10}
Runtime

## Sigmoid kernel

In [27]:
grid = {'sigma_n': [0.05, 0.1, 1, 5, 10],
        'alpha': [0.05, 1, 5]}
grid_list = list(ParameterGrid(grid))
grid_list

[{'alpha': 0.05, 'sigma_n': 0.05},
 {'alpha': 0.05, 'sigma_n': 0.1},
 {'alpha': 0.05, 'sigma_n': 1},
 {'alpha': 0.05, 'sigma_n': 5},
 {'alpha': 0.05, 'sigma_n': 10},
 {'alpha': 1, 'sigma_n': 0.05},
 {'alpha': 1, 'sigma_n': 0.1},
 {'alpha': 1, 'sigma_n': 1},
 {'alpha': 1, 'sigma_n': 5},
 {'alpha': 1, 'sigma_n': 10},
 {'alpha': 5, 'sigma_n': 0.05},
 {'alpha': 5, 'sigma_n': 0.1},
 {'alpha': 5, 'sigma_n': 1},
 {'alpha': 5, 'sigma_n': 5},
 {'alpha': 5, 'sigma_n': 10}]

In [14]:
for case in grid_list:

    model = GP_Regression_SOR(K=SigmoidKernel(alpha=case['alpha']), sigma_n=case['sigma_n'], device=device, B=0.1)
    start_time = datetime.now()
    model.fit(X_train, Y_train, X_valid)
    pred = model.predict()
    runtime = datetime.now() - start_time
    mse = nn.MSELoss()(pred, Y_valid).item()
    
    print(case)
    print('Runtime:: ', runtime.microseconds)
    print('MSE loss: ', mse)
    print('')

{'alpha': 0.05, 'sigma_n': 0.05}
Runtime::  861084
MSE loss:  8.259437873146231

{'alpha': 0.05, 'sigma_n': 0.1}
Runtime::  954705
MSE loss:  7.292323144034025

{'alpha': 0.05, 'sigma_n': 1}
Runtime::  227890
MSE loss:  7.260592703719794

{'alpha': 0.05, 'sigma_n': 5}
Runtime::  271570
MSE loss:  10.048423248229945

{'alpha': 0.05, 'sigma_n': 10}
Runtime::  775042
MSE loss:  116.71605128861682

{'alpha': 1, 'sigma_n': 0.05}
Runtime::  964304
MSE loss:  26.81745081897477

{'alpha': 1, 'sigma_n': 0.1}
Runtime::  149503
MSE loss:  36.594688592294084

{'alpha': 1, 'sigma_n': 1}
Runtime::  240528
MSE loss:  32.434234757676535

{'alpha': 1, 'sigma_n': 5}
Runtime::  919076
MSE loss:  32.37575823305953

{'alpha': 1, 'sigma_n': 10}
Runtime::  872059
MSE loss:  37.32291690285767

{'alpha': 5, 'sigma_n': 0.05}
Runtime::  17034
MSE loss:  34.774999357897315

{'alpha': 5, 'sigma_n': 0.1}
Runtime::  819889
MSE loss:  32.8562655970894

{'alpha': 5, 'sigma_n': 1}
Runtime::  965240
MSE loss:  41.359450

In [28]:
# cross-validation

for case in grid_list:

    X, Y = X_train, Y_train
    KF = KFold(n_splits=5, shuffle=True, random_state=2022)
    mse_list = []
    start_time = datetime.now()
    for train_idx, valid_idx in KF.split(X):
        X_train, Y_train = X[train_idx], Y[train_idx]
        X_valid, Y_valid = X[valid_idx], Y[valid_idx]
        # model
        model = GP_Regression_SOR(K=SigmoidKernel(alpha=case['alpha']), sigma_n=case['sigma_n'], device=device, B=0.1)
        model.fit(X_train, Y_train, X_valid)
        pred = model.predict()
        mse = nn.MSELoss()(pred, Y_valid).item()
        gc.collect(); torch.cuda.empty_cache() # clear memory
        mse_list.append(mse)
    final_loss = np.mean(mse_list)
    runtime = datetime.now() - start_time

    print(case)
    print('Runtime:: ', runtime.microseconds)
    print(final_loss)
    print('')


{'alpha': 0.05, 'sigma_n': 0.05}
Runtime::  428384
347.4473421766501

{'alpha': 0.05, 'sigma_n': 0.1}
Runtime::  323127
425.1552264930798

{'alpha': 0.05, 'sigma_n': 1}
Runtime::  324164
372.8934757857834

{'alpha': 0.05, 'sigma_n': 5}
Runtime::  315283
415.6811903484428

{'alpha': 0.05, 'sigma_n': 10}
Runtime::  318287
763.9305913289479

{'alpha': 1, 'sigma_n': 0.05}
Runtime::  329295
392.9725012705678

{'alpha': 1, 'sigma_n': 0.1}
Runtime::  328295
70142.58243024274

{'alpha': 1, 'sigma_n': 1}
Runtime::  319287
384.1036544636424

{'alpha': 1, 'sigma_n': 5}
Runtime::  328567
349.19790351265203

{'alpha': 1, 'sigma_n': 10}
Runtime::  320555
393.24389765075546

{'alpha': 5, 'sigma_n': 0.05}
Runtime::  315398
349.5109498661801

{'alpha': 5, 'sigma_n': 0.1}
Runtime::  305277
573.7738458207726

{'alpha': 5, 'sigma_n': 1}
Runtime::  292219
521.9446438603425

{'alpha': 5, 'sigma_n': 5}
Runtime::  297566
564.6281247663621

{'alpha': 5, 'sigma_n': 10}
Runtime::  291098
522.1914634517682



# 3. Other regression model

In [15]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

X, y = X_train, Y_train
start_time = datetime.now()
model = MultiOutputRegressor(Ridge(random_state=2022)).fit(X, y)
pred = model.predict(X_test)
runtime = datetime.now() - start_time
mse = mean_squared_error(pred, Y_test).item()

print('Runtime: ', runtime.microseconds)
print('MSE loss: ', mse)

Runtime:  200181
MSE loss:  10.719762261306766


# HPO - NLL optimizer (discarded)

In [48]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.00002)
n_iter=50
print(f'Initial Parameter: {list(model.parameters())[0].item()}')
print('')
for i in range(n_iter):
    print(f'[Iteration {i}]')
    nll = model.calculate_NLL()
    optimizer.zero_grad()
    nll.backward() # calculate derivatives
    optimizer.step() # update parameters
    print(f"NLL: {nll.item()}")
    for i in range(len(list(model.parameters()))):
        print('Paremeter: ', list(model.parameters())[i].item())
    pred = model.predict()
    mse = nn.MSELoss()(pred, Y_test).item()
    print(f"MSE:", mse)
    print("")


Initial Parameter: 0.05000000074505806

[Iteration 0]
NLL: 970830.0
Paremeter:  18.334877014160156
MSE: 376.5684566220583

[Iteration 1]
NLL: inf
Paremeter:  nan
MSE: 376.5684566220583

[Iteration 2]
NLL: nan
Paremeter:  nan
MSE: 376.5684566220583

[Iteration 3]
NLL: nan
Paremeter:  nan
MSE: 376.5684566220583

[Iteration 4]
NLL: nan
Paremeter:  nan
MSE: 376.5684566220583

[Iteration 5]
NLL: nan
Paremeter:  nan
MSE: 376.5684566220583

[Iteration 6]
NLL: nan
Paremeter:  nan
MSE: 376.5684566220583

[Iteration 7]
NLL: nan
Paremeter:  nan
MSE: 376.5684566220583

[Iteration 8]
NLL: nan
Paremeter:  nan


KeyboardInterrupt: 